In [58]:
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.geography_utils import CA_NAD83Albers_m, WGS84
gcsgp = GCSGeoPandas()
from shared_utils import gtfs_utils_v2
from shared_utils.rt_utils import show_full_df

import utils

# Special Handling for SF - Emeryville Route

* Check that trips in SJJPA and CC feeds do not overlap

In [2]:
analysis_date = '2025-08-18'

In [3]:
ca = gcsgp.read_parquet('gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/ca_boundary.parquet')

## New SJJPA San Joaquins feed

In [8]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

In [9]:
sanj = feeds.query('name.str.contains("Amtrak San Joaquins")')
sanj

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,type,regional_feed_type
82,a04bf510313ab925aa37eea637a18bec,2025-08-18,d9e1e77d0754b712fc608741ae3836f5,America/Los_Angeles,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,524ea6209600e9a2de34a02cf9068729,Amtrak San Joaquins Schedule,schedule,None


In [12]:
sanj_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=sanj.feed_key, shape_cols=utils.shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [16]:
sanj_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=sanj.feed_key, trip_cols=utils.trip_cols)

In [62]:
sanj_stops = gtfs_utils_v2.get_stops(analysis_date, operator_feeds=sanj.feed_key, stop_cols=utils.stop_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [59]:
sanj_st = gtfs_utils_v2.get_stop_times(analysis_date, operator_feeds=sanj.feed_key, get_df=True,
                                      trip_df = sanj_trips, stop_time_cols=utils.stop_time_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


In [64]:
sanj_sf_emy = sanj_trips.query('route_short_name == "Route 99"')

In [128]:
import importlib
importlib.reload(utils)

<module 'utils' from '/home/jovyan/data-analyses/thruway_intercity_bus/utils.py'>

In [67]:
sanj_st_merged = sanj_stops.merge(sanj_st, on = ['feed_key', 'stop_id']).merge(sanj_sf_emy, on = ['feed_key', 'trip_id'])

In [69]:
sanj_st_merged.shape

(20, 31)

In [94]:
# utils.format_stop_times(sanj_st_merged)

## Capitol Corridor

In [22]:
cc_feed = feeds.query('name.str.contains("Capitol")')

In [23]:
cc_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=cc_feed.feed_key, shape_cols=shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [24]:
cc_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=cc_feed.feed_key, trip_cols=trip_cols)

In [70]:
# cc_trips[['route_short_name', 'route_long_name', 'trip_instance_key']].groupby(['route_short_name', 'route_long_name']).count()

In [71]:
cc_stops = gtfs_utils_v2.get_stops(analysis_date, operator_feeds=cc_feed.feed_key, stop_cols=utils.stop_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [72]:
cc_st = gtfs_utils_v2.get_stop_times(analysis_date, operator_feeds=cc_feed.feed_key, get_df=True,
                                      trip_df = cc_trips, stop_time_cols=utils.stop_time_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


In [78]:
cc_sf_emy = cc_trips.query('route_short_name == "SF"')

In [79]:
cc_st_merged = cc_stops.merge(cc_st, on = ['feed_key', 'stop_id']).merge(cc_sf_emy, on = ['feed_key', 'trip_id'])

In [80]:
cc_st_merged.shape

(52, 31)

In [93]:
# utils.format_stop_times(cc_st_merged)

In [140]:
both_st_merged = pd.concat([sanj_st_merged, cc_st_merged])

In [141]:
both_trip_ix = both_st_merged[['trip_id', 'direction_id', 'arrival_sec']].groupby(['trip_id', 'direction_id']).min().reset_index().sort_values(['direction_id', 'arrival_sec']).reset_index(drop=True)

In [142]:
both_trip_ix['sort_trip_id'] = range(both_trip_ix.shape[0])
both_trip_ix = both_trip_ix.drop(columns=['arrival_sec'])

In [143]:
both_st_merged = both_st_merged.merge(both_trip_ix, on=['trip_id', 'direction_id'])

### Merged Timetable

https://www.capitolcorridor.org//trainschedule/Train_Schedules.pdf?v=05082025

https://amtraksanjoaquins.com/route-99-oakland-emeryville-san-francisco/

* CC trip_id 526E is probably incorrect in GTFS and should be 7:55a SF -> 8:25a EMY (see pdf schedule for train 526)
* CC trip_id 524E and SJ trip_id 6610 seem like two buses going to the same place at about the same time... do we need the capacity and could they be consolidated to one run?
* other CC and SJ trips are spaced out enough that consolidation would slow travel times, especially vs BART connection at Richmond

In [144]:
show_full_df(utils.format_stop_times(both_st_merged).sort_values(['direction_id', 'sort_trip_id', 'arrival_sec']))

,route_short_name,route_long_name,direction_id,stop_id,stop_name,trip_id,stop_sequence,arrival_time,departure_time,route_id,name,arrival_sec,sort_trip_id,geometry
63,SF,Shuttle_SF,0,74732,San Francisco - Transbay Terminal,524E,0,6:55:00,6:55:00,SF,Bay Area 511 Capitol Corridor Schedule,24900,0,POINT (-122.39659 37.79021)
62,SF,Shuttle_SF,0,74369,Emeryville,524E,1,7:25:00,7:25:00,SF,Bay Area 511 Capitol Corridor Schedule,26700,0,POINT (-122.29181 37.84042)
61,SF,Shuttle_SF,0,74732,San Francisco - Transbay Terminal,526E,0,6:55:00,6:55:00,SF,Bay Area 511 Capitol Corridor Schedule,24900,1,POINT (-122.39659 37.79021)
60,SF,Shuttle_SF,0,74369,Emeryville,526E,1,7:25:00,7:25:00,SF,Bay Area 511 Capitol Corridor Schedule,26700,1,POINT (-122.29181 37.84042)
17,Route 99,Oakland-Emeryville-San Francisco,0,bSFC,San Francisco - Transbay Terminal Bus Stop,6610,0,7:00:00,7:00:00,99,Amtrak San Joaquins Schedule,25200,2,POINT (-122.39850 37.78850)
16,Route 99,Oakland-Emeryville-San Francisco,0,bEMY,Emeryville Ca Station Bus Stop,6610,1,7:30:00,7:30:00,99,Amtrak San Joaquins Schedule,27000,2,POINT (-122.29181 37.84045)
19,Route 99,Oakland-Emeryville-San Francisco,0,bSFC,San Francisco - Transbay Terminal Bus Stop,6612_w,0,9:00:00,9:00:00,99,Amtrak San Joaquins Schedule,32400,3,POINT (-122.39850 37.78850)
18,Route 99,Oakland-Emeryville-San Francisco,0,bEMY,Emeryville Ca Station Bus Stop,6612_w,1,9:30:00,9:30:00,99,Amtrak San Joaquins Schedule,34200,3,POINT (-122.29181 37.84045)
71,SF,Shuttle_SF,0,74732,San Francisco - Transbay Terminal,528E,0,9:25:00,9:25:00,SF,Bay Area 511 Capitol Corridor Schedule,33900,4,POINT (-122.39659 37.79021)
70,SF,Shuttle_SF,0,74369,Emeryville,528E,1,9:55:00,9:55:00,SF,Bay Area 511 Capitol Corridor Schedule,35700,4,POINT (-122.29181 37.84042)
